In [1]:
import math

def findGCD(seq):
    gcd = seq[0]
    for i in range(1,len(seq)):
        gcd=math.gcd(gcd, seq[i])
    return gcd

In [2]:
def findSignature(seq):
    nonzero_seq = [d for d in seq if d!=0]
    if len(nonzero_seq)==0:
        return seq
    sign = 1 if nonzero_seq[0]>0 else -1
    gcd = findGCD(seq)
    return [sign*x//gcd for x in seq]

In [3]:
def findDerivative(seq):
    return [0] if len(seq)<=1 else [seq[i]-seq[i-1] for i in range(1,len(seq))]

In [4]:
def addAll(seq, node, list):
    if 'value' in node:
        list.append( ( seq, node['value'] ) )
    for key in node:
        if key != 'value':
            addAll(seq + [key], node[key], list)


In [5]:
class prefixTree:
    def __init__(self):
        self.data={}
        self.puts=0
        self.nodes=0
    
    def put(self, seq, value):
        node=self.data
        nodeCreated=False
        for i in range(0,len(seq)):
            item=seq[i]
            if not item in node:
                node[item]={}
                if 'value' in node:
                    del node['value']
                self.nodes+=1
                nodeCreated=True
            node=node[item]
        if nodeCreated:
            node['value']=value
            self.puts+=1
        elif 'value' in node:
            node['value']=max(node['value'], value)
    
    def prefix(self, seq):
        list=[]
        node=self.data
        for i in range(0,len(seq)):
            item=seq[i]
            if item in node:
                node=node[item]
            else:
                return list
        addAll(seq, node, list)
        return list
    
    def hasPrefix(self, seq):
        node=self.data
        for i in range(0,len(seq)):
            item=seq[i]
            if item in node:
                node=node[item]
            else:
                return False
        return True

In [6]:
from functools import reduce
next_elm = []

def findNext(seq, trie):
    while True:
        nonZeroIndex =-1
        for i in range(0,len(seq)):
            if seq[i] != 0: 
                nonZeroIndex = i
                break
        if nonZeroIndex < 0:
            return 0
        signature = findSignature(seq)
        list = trie.prefix( signature )
        list = filter(lambda x: len(x[0])>len(signature), list)
        item = next(list, None)
        if item != None:
            best = reduce(lambda a, b: a if a[1]>b[1] else b if b[1]>a[1] else a if len(b[0])<=len(a[0]) else b, list, item)
            nextElement = best[0][len(seq)]
            nextElement *= seq[nonZeroIndex]//signature[nonZeroIndex]
            return nextElement
        if len(seq) <= 3: 
            break
        seq = seq[1:]
    return None

def findNextAndDerive(seq, trie):
    nextElement=findNext(seq, trie)
    if nextElement==None:
        der=findDerivative(seq)
        if len(der)<=3:
            return None
        nextElement=findNextAndDerive(der, trie)
        if nextElement==None:
            return None
        return seq[len(seq)-1]+nextElement
    return nextElement


In [7]:
##### import datasets #####

import pandas as pd

#train_df= pd.read_csv('./data/train.csv', index_col="Id", nrows=100)
test_df = pd.read_csv('/home/nastya/Desktop/uData/data/test.csv', index_col="Id")
train_df= pd.read_csv('/home/nastya/Desktop/uData/data/train.csv', index_col="Id")

train_df = train_df['Sequence'].to_dict()
test_df = test_df['Sequence'].to_dict()
#seq_train = {0: [1 for x in range(0,400)]}
seq_train = {}
seq_test = {}
#seq_test = {0: [1 for x in range(0,400)]}

for key in train_df:
    seq1 = train_df[key]
    seq1 = [int(x) for x in seq1.split(',')]
    seq_train[key] = seq1

for key in test_df:
    seq2 = test_df[key]
    seq2 = [int(x) for x in seq2.split(',')]
    seq_test[key] = seq2
    
### our data is dict ###

In [ ]:
### creating trie for test data
## not run building two trie together
import json
trieTest = prefixTree()
if True:
    for id in seq_test:
        der_test = seq_test[id]
        for derAttempts in range(4):
            test_seq = der_test
            firstInTrie = False
            for subseqAttempts in range(4-derAttempts):
                while len(test_seq)>0 and test_seq[0] == 0:
                    test_seq = test_seq[1:]
                signature = findSignature(test_seq)
                if trieTest.hasPrefix( signature ):
                    if subseqAttempts == 0:
                        firstInTrie = True
                    break
                trieTest.put( signature, len(test_seq)*100//len(der_test) )
                if len(test_seq) <= 3:
                    break
                test_seq = test_seq[1:]
            if firstInTrie:
                break
            der_test = findDerivative(der_test)

In [8]:
### creating trie for train data
trieTrain = prefixTree()
if True:
    for id in seq_train:
        der_train = seq_train[id]
        for derAttempts in range(4):
            train_seq = der_train
            firstInTrie = False
            for subseqAttempts in range(4-derAttempts):
                while len(train_seq)>0 and train_seq[0] == 0:
                    train_seq = train_seq[1:]
                signature = findSignature(train_seq)
                if trieTrain.hasPrefix( signature ):
                    if subseqAttempts == 0:
                        firstInTrie = True
                    break
                trieTrain.put( signature, len(train_seq)*100//len(der_train) )#задаем веса для каждого узла len(train_seq)*100//len(der_train)
                if len(train_seq) <= 3:
                    break
                train_seq = train_seq[1:]
            if firstInTrie:
                break
            der_train = findDerivative(der_train)


In [ ]:
#### accuracy for test dataset
total=0
guessed=0  
for key in test_df:
    data_pred_list = list(map(int, test_df[key].split(',')))
    data_pred = data_pred_list[0:len(data_pred_list)-1]
    tagret = data_pred_list[-1]
    next_elm = findNextAndDerive(data_pred,trieTest)   
    total += 1
    if next_elm == None:
        next_elm = 0
    else:
        if next_elm == tagret:
            guessed += 1
            
print('Total %d' %total)
print('Guessed %d' %guessed)
print('Percent %d' %int(guessed*100//total))


In [9]:
total=0
guessed=0
with open('/home/nastya/Desktop/uData/data/result_prefic_trie.csv', 'w+') as output:
    output.write('"Id","Last"\n')    
    for id in train_df:
        der = list(map(int, train_df[id].split(',')))
        nextElement = findNextAndDerive(der, trieTrain)
        output.write(str(id))
        output.write(',')
        total += 1
        if nextElement == None:
            output.write('0')
        else:
            output.write(str(nextElement))
            guessed+=1
        output.write('\n')
        
print('Total %d' %total)
print('Guessed %d' %guessed)
print('Percent %d' %int(guessed*100//total))

Total 113845
Guessed 51331
Percent 45
